<a href="https://colab.research.google.com/github/ash-Tech07/Automatic_User_Review_Tagging/blob/main/Automatic_user_review_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
>>> import nltk
>>> nltk.download()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/Colab Notebooks/


**Importing required libraries for the training**

In [ ]:
import re
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
import keras
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import math
import nltk

**1. Loading the User review dataset**

In [ ]:
data = pd.read_csv('output.csv')
data

,rating,date,variation,review,feedback
0,5,31-Jul-18,Charcoal Fabric,Love my Echo!,positive
1,5,31-Jul-18,Charcoal Fabric,Loved it!,positive
2,4,31-Jul-18,Walnut Finish,"Sometimes while playing a game, you can answer...",positive
3,5,31-Jul-18,Charcoal Fabric,I have had a lot of fun with this thing. My 4 ...,positive
4,5,31-Jul-18,Charcoal Fabric,Music,positive
...,...,...,...,...,...
3145,5,30-Jul-18,Black Dot,"Perfect for kids, adults and everyone in betwe...",positive
3146,5,30-Jul-18,Black Dot,"Listening to music, searching locations, check...",positive
3147,5,30-Jul-18,Black Dot,"I do love these things, i have them running my...",positive
3148,5,30-Jul-18,White Dot,Only complaint I have is that the sound qualit...,positive


**2. Data Preprocessing**

In [ ]:
CLEANR = re.compile('<.*?>') 
def remove_tags(st):
    cleantext = re.sub(CLEANR, '', st)
    cleantext = re.sub('http://\S+|https://\S+', '', cleantext)
    cleantext = re.sub('[^A-Za-z0-9 ]+', '', cleantext)
    return cleantext
data['review']=data['review'].apply(lambda w : remove_tags(w))
data

,rating,date,variation,review,feedback
0,5,31-Jul-18,Charcoal Fabric,Love my Echo,positive
1,5,31-Jul-18,Charcoal Fabric,Loved it,positive
2,4,31-Jul-18,Walnut Finish,Sometimes while playing a game you can answer ...,positive
3,5,31-Jul-18,Charcoal Fabric,I have had a lot of fun with this thing My 4 y...,positive
4,5,31-Jul-18,Charcoal Fabric,Music,positive
...,...,...,...,...,...
3145,5,30-Jul-18,Black Dot,Perfect for kids adults and everyone in between,positive
3146,5,30-Jul-18,Black Dot,Listening to music searching locations checkin...,positive
3147,5,30-Jul-18,Black Dot,I do love these things i have them running my ...,positive
3148,5,30-Jul-18,White Dot,Only complaint I have is that the sound qualit...,positive


In [ ]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
data['review'] = data['review'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
data

,rating,date,variation,review,feedback
0,5,31-Jul-18,Charcoal Fabric,Love Echo,positive
1,5,31-Jul-18,Charcoal Fabric,Loved,positive
2,4,31-Jul-18,Walnut Finish,Sometimes playing game answer question correct...,positive
3,5,31-Jul-18,Charcoal Fabric,I lot fun thing My 4 yr old learns dinosaurs c...,positive
4,5,31-Jul-18,Charcoal Fabric,Music,positive
...,...,...,...,...,...
3145,5,30-Jul-18,Black Dot,Perfect kids adults everyone,positive
3146,5,30-Jul-18,Black Dot,Listening music searching locations checking t...,positive
3147,5,30-Jul-18,Black Dot,I love things running entire home TV lights th...,positive
3148,5,30-Jul-18,White Dot,Only complaint I sound quality isnt great I mo...,positive


In [ ]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()
def lemmatize_text(text):
    st = ""
    for w in w_tokenizer.tokenize(text):
        st = st + lemmatizer.lemmatize(w) + " "
    return st
data['review'] = data.review.apply(lemmatize_text)
data

,rating,date,variation,review,feedback
0,5,31-Jul-18,Charcoal Fabric,Love Echo,positive
1,5,31-Jul-18,Charcoal Fabric,Loved,positive
2,4,31-Jul-18,Walnut Finish,Sometimes playing game answer question correct...,positive
3,5,31-Jul-18,Charcoal Fabric,I lot fun thing My 4 yr old learns dinosaur co...,positive
4,5,31-Jul-18,Charcoal Fabric,Music,positive
...,...,...,...,...,...
3145,5,30-Jul-18,Black Dot,Perfect kid adult everyone,positive
3146,5,30-Jul-18,Black Dot,Listening music searching location checking ti...,positive
3147,5,30-Jul-18,Black Dot,I love thing running entire home TV light ther...,positive
3148,5,30-Jul-18,White Dot,Only complaint I sound quality isnt great I mo...,positive


In [ ]:
s = 0.0
for i in data['review']:
    word_list = i.split()
    s = s + len(word_list)
print("Average length of each review : ",s/data.shape[0])
pos = 0
for i in range(data.shape[0]):
    if data.iloc[i]['feedback'] == 'positive':
        pos = pos + 1
neg = data.shape[0]-pos
print("Percentage of reviews with positive sentiment is "+str(pos/data.shape[0]*100)+"%")
print("Percentage of reviews with negative sentiment is "+str(neg/data.shape[0]*100)+"%")

Average length of each review :  14.604761904761904
Percentage of reviews with positive sentiment is 91.84126984126985%
Percentage of reviews with negative sentiment is 8.158730158730158%


**3. Encoding Labels and Making Train-Test Split list**

In [ ]:
reviews = data['review'].values
labels = data['feedback'].values
encoder = LabelEncoder()
encoded_labels = encoder.fit_transform(labels)

In [ ]:
train_sentences, test_sentences, train_labels, test_labels = train_test_split(reviews, encoded_labels, stratify = encoded_labels)

In [ ]:
# Hyperparameters of the model
vocab_size = 3000 # choose based on statistics
oov_tok = ''
embedding_dim = 100
max_length = 200 # choose based on statistics, for example 150 to 200
padding_type='post'
trunc_type='post'
# tokenize sentences
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index
# convert train dataset to sequence and pad sequences
train_sequences = tokenizer.texts_to_sequences(train_sentences)
train_padded = pad_sequences(train_sequences, padding='post', maxlen=max_length)
# convert Test dataset to sequence and pad sequences
test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_sequences, padding='post', maxlen=max_length)

**4. Building the Model**

In [ ]:
# model initialization
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),
    keras.layers.Dense(24, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])
# compile model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
# model summary
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 200, 100)          300000    
                                                                 
 bidirectional_2 (Bidirectio  (None, 128)              84480     
 nal)                                                            
                                                                 
 dense_4 (Dense)             (None, 24)                3096      
                                                                 
 dense_5 (Dense)             (None, 1)                 25        
                                                                 
Total params: 387,601
Trainable params: 387,601
Non-trainable params: 0
_________________________________________________________________


**5. Model Training and Evaluation**

In [ ]:
num_epochs = 5
history = model.fit(train_padded, train_labels, 
                    epochs=num_epochs, verbose=1, 
                    validation_split=0.1)

Epoch 1/5
67/67 [==============================] - 5s 31ms/step - loss: 0.3276 - accuracy: 0.9134 - val_loss: 0.2526 - val_accuracy: 0.9198
Epoch 2/5
67/67 [==============================] - 1s 19ms/step - loss: 0.2116 - accuracy: 0.9247 - val_loss: 0.1752 - val_accuracy: 0.9409
Epoch 3/5
67/67 [==============================] - 2s 26ms/step - loss: 0.0930 - accuracy: 0.9680 - val_loss: 0.1940 - val_accuracy: 0.9536
Epoch 4/5
67/67 [==============================] - 2s 26ms/step - loss: 0.0416 - accuracy: 0.9854 - val_loss: 0.2436 - val_accuracy: 0.9620
Epoch 5/5
67/67 [==============================] - 1s 19ms/step - loss: 0.0257 - accuracy: 0.9915 - val_loss: 0.2704 - val_accuracy: 0.9662


In [ ]:
prediction = model.predict(test_padded)
# Get labels based on probability 1 if p>= 0.5 else 0
pred_labels = []
for i in prediction:
    if i >= 0.5:
        pred_labels.append(1)
    else:
        pred_labels.append(0)
print("Accuracy of prediction on test set : ", accuracy_score(test_labels,pred_labels))

25/25 [==============================] - 1s 10ms/step
Accuracy of prediction on test set :  0.932741116751269


**6. Using the model to determine the tag of alexa user reviews**

In [ ]:
# reviews on which we need to predict
sentence = ["The tv is very good and had high qualilty", 
            "This mobile is the worst of all", 
            "The lamp has poor structure but good light intensity"]
# convert to a sequence
sequences = tokenizer.texts_to_sequences(sentence)
# pad the sequence
padded = pad_sequences(sequences, padding='post', maxlen=max_length)
# Get labels based on probability 1 if p>= 0.5 else 0
prediction = model.predict(padded)
pred_labels = []
for i in prediction:
    if i >= 0.5:
        pred_labels.append(1)
    else:
        pred_labels.append(0)
for i in range(len(sentence)):
    print(sentence[i])
    if pred_labels[i] == 1:
        s = 'Positive'
    else:
        s = 'Negative'
    print("Predicted sentiment : ",s)

1/1 [==============================] - 0s 22ms/step
The tv is very good and had high qualilty
Predicted sentiment :  Positive
This mobile is the worst of all
Predicted sentiment :  Negative
The lamp has poor structure but good light intensity
Predicted sentiment :  Positive
